# SLC4A1 Bruce et al., 1997. Distal renal tubular acidosis 1 	179800 

Data derived from [Bruce LJ (1997) Familial distal renal tubular acidosis is associated with mutations in the red cell anion exchanger (Band 3, AE1) gene. J Clin Invest 100:1693-707. PMID: 9312167](https://pubmed.ncbi.nlm.nih.gov/9312167/)

Variants were encoded as follows

-  family A: G->A mutation at 1766 (R589H) - NM_000342.4(SLC4A1):c.1766G>A (p.Arg589His)
-  family B: G->A mutation at 1766 (R589H) - NM_000342.4(SLC4A1):c.1766G>A (p.Arg589His)
-  family C: C->T mutation at 1765 (R589C) - NM_000342.4(SLC4A1):c.1765C>T (p.Arg589Cys)
-  family D: C->T mutation at 1838 (S613F) - NM_000342.4(SLC4A1):c.1838C>T (p.Ser613Phe) 

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import PhenopacketTable, QcVisualizer
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.15


In [2]:
PMID="PMID:9312167"
title = "Familial distal renal tubular acidosis is associated with mutations in the red cell anion exchanger (Band 3, AE1) gene"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2023-10-09


In [3]:
df = pd.read_csv("input/bruce_SLC4A1.tsv", sep='\t')

In [4]:
df.head()

,individual,Sex/Age,Creatinine(plasma),HCO3(plasma),K(Plasma),min_pH(urine),maximumOsm(urine),Age,nephrocalcinosis,kidney stones,dRTA,raised hemoglobin,raised hematocrti,NM_000342.4
0,A:I:1,F/60,74,24,4.1,6.26,635,74,No,Yes,Yes,No,No,c.1766G>A
1,A:I:2,F/40,86,18,3.4,6.35,572,71,Yes,Yes,No,No,No,c.1766G>A
2,A:I:3,F/44,177,14,4.0,6.6,n.t.,died 60,Yes,No,Yes,No,No,c.1766G>A
3,A:II:3,M/27,115,23,3.1,6.45,525,47,Yes,Yes,Yes,No,No,c.1766G>A
4,A:II:5,F/35,88,25,3.7,6.09,855,35,No,No,Yes,No,No,c.1766G>A


In [5]:
generator = SimpleColumnMapperGenerator(df=df, hpo_cr=hpo_cr, observed="Yes", excluded="No")
column_mapper_d = generator.try_mapping_columns()
display(HTML(generator.to_html()))

Result,Columns
Mapped,nephrocalcinosis; kidney stones
Unmapped,individual; Sex/Age; Creatinine(plasma); HCO3(plasma); K(Plasma); min_pH(urine); maximumOsm(urine); Age; dRTA; raised hemoglobin; raised hematocrti; NM_000342.4


In [6]:
# Creatinine(plasma). Normal 50–125
# Elevated circulating creatinine concentration HP:0003259

creaTM = ThresholdedColumnMapper(hpo_id="HP:0003259", 
                                 hpo_label="Elevated circulating creatinine concentration",
                                threshold=125,
                                call_if_above=True)
#creaTM.preview_column(df["Creatinine(plasma)"])
column_mapper_d["Creatinine(plasma)"] = creaTM

In [7]:
# HCO3(plasma) 22–31 
# Decreased serum bicarbonate concentration HP:0032066
bicarbTM = ThresholdedColumnMapper(hpo_id="HP:0032066", 
                                 hpo_label="Decreased serum bicarbonate concentration",
                                threshold=22,
                                call_if_above=False)
bicarbTM.preview_column(df["HCO3(plasma)"])
column_mapper_d["HCO3(plasma)"] = bicarbTM

In [8]:
# K (plasma): 3.4–4.8 
# Hypokalemia HP:0002900
kTM = ThresholdedColumnMapper(hpo_id="HP:0002900", 
                                 hpo_label="Hypokalemia",
                                threshold=3.4,
                                call_if_above=False)
kTM.preview_column(df["K(Plasma)"])
column_mapper_d["K(Plasma)"] = kTM

In [9]:
# min_pH(urine) < 5.30
# "HP:0032944", "Alkaline urine",
alkU_TM = ThresholdedColumnMapper(hpo_id="HP:0032944", 
                                 hpo_label="Alkaline urine",
                                threshold=5.3,
                                call_if_above=True)
alkU_TM.preview_column(df["min_pH(urine)"])
column_mapper_d["min_pH(urine)"] = alkU_TM

In [10]:
# maximumOsm(urine) >  940
# Impaired renal concentrating ability HP:0004727
concTM = ThresholdedColumnMapper(hpo_id="HP:0004727", 
                                 hpo_label="Impaired renal concentrating ability",
                                threshold=940,
                                call_if_above=False)
concTM.preview_column(df["maximumOsm(urine)"])
column_mapper_d["maximumOsm(urine)"] = concTM

In [11]:
# dRTA
#  HP:0008341 Distal renal tubular acidosis
dRTA = SimpleColumnMapper(hpo_id="HP:0008341", hpo_label="Distal renal tubular acidosis", observed="Yes", excluded="No")
#dRTA.preview_column(df["dRTA"])
column_mapper_d["dRTA"] = dRTA

In [12]:
df["Sex"] =  df["Sex/Age"].apply(lambda x: x.split("/")[0])
df["Age"] =  df["Sex/Age"].apply(lambda x: x.split("/")[1])
ageMapper = AgeColumnMapper.by_year(column_name="Age")
sexMapper = SexColumnMapper(male_symbol="M", female_symbol="F", column_name="Sex")

In [13]:
SLC4A1_transcript = "NM_000342.4"
vvalidator = VariantValidator(genome_build="hg38", transcript=SLC4A1_transcript)
var_d = {}
for v in df["NM_000342.4"].unique():
    var = vvalidator.encode_hgvs(v)
    var_d[v] = var
varMapper = VariantColumnMapper(variant_column_name="NM_000342.4", variant_d=var_d, default_genotype="heterozygous")

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000342.4%3Ac.1766G>A/NM_000342.4?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000342.4%3Ac.1765C>T/NM_000342.4?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000342.4%3Ac.1838C>T/NM_000342.4?content-type=application%2Fjson


In [14]:
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_mapper_d, 
                        individual_column_name="individual", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper,
                        metadata=metadata)
 	 
rta1 = Disease(disease_id='OMIM:179800', disease_label='Distal renal tubular acidosis 1')
encoder.set_disease(rta1)

In [15]:
individuals = encoder.get_individuals()

In [17]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_html()))

In [18]:
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
A:I:1 (FEMALE; P60Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrolithiasis (HP:0000787); Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); Distal renal tubular acidosis (HP:0008341); excluded: Nephrocalcinosis (HP:0000121); excluded: Elevated circulating creatinine concentration (HP:0003259); excluded: Decreased serum bicarbonate concentration (HP:0032066); excluded: Hypokalemia (HP:0002900)
A:I:2 (FEMALE; P40Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrocalcinosis (HP:0000121); Nephrolithiasis (HP:0000787); Decreased serum bicarbonate concentration (HP:0032066); Hypokalemia (HP:0002900); Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); excluded: Elevated circulating creatinine concentration (HP:0003259); excluded: Distal renal tubular acidosis (HP:0008341)
A:I:3 (FEMALE; P44Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrocalcinosis (HP:0000121); Elevated circulating creatinine concentration (HP:0003259); Decreased serum bicarbonate concentration (HP:0032066); Alkaline urine (HP:0032944); Distal renal tubular acidosis (HP:0008341); excluded: Nephrolithiasis (HP:0000787); excluded: Hypokalemia (HP:0002900)
A:II:3 (MALE; P27Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrocalcinosis (HP:0000121); Nephrolithiasis (HP:0000787); Hypokalemia (HP:0002900); Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); Distal renal tubular acidosis (HP:0008341); excluded: Elevated circulating creatinine concentration (HP:0003259); excluded: Decreased serum bicarbonate concentration (HP:0032066)
A:II:5 (FEMALE; P35Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); Distal renal tubular acidosis (HP:0008341); excluded: Nephrocalcinosis (HP:0000121); excluded: Nephrolithiasis (HP:0000787); excluded: Elevated circulating creatinine concentration (HP:0003259); excluded: Decreased serum bicarbonate concentration (HP:0032066); excluded: Hypokalemia (HP:0002900)
A:II:6 (MALE; P36Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrocalcinosis (HP:0000121); Decreased serum bicarbonate concentration (HP:0032066); Hypokalemia (HP:0002900); Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); Distal renal tubular acidosis (HP:0008341); excluded: Nephrolithiasis (HP:0000787); excluded: Elevated circulating creatinine concentration (HP:0003259)
A:II:8 (MALE; P34Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrocalcinosis (HP:0000121); Nephrolithiasis (HP:0000787); Elevated circulating creatinine concentration (HP:0003259); Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); Distal renal tubular acidosis (HP:0008341); excluded: Decreased serum bicarbonate concentration (HP:0032066); excluded: Hypokalemia (HP:0002900)
A:II:9 (MALE; P31Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrolithiasis (HP:0000787); Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); Distal renal tubular acidosis (HP:0008341); excluded: Nephrocalcinosis (HP:0000121); excluded: Elevated circulating creatinine concentration (HP:0003259); excluded: Decreased serum bicarbonate concentration (HP:0032066); excluded: Hypokalemia (HP:0002900)
A:III:1 (MALE; P9Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Decreased serum bicarbonate concentration (HP:0032066); Alkaline urine (HP:0032944); Distal renal tubular acidosis (HP:0008341); excluded: Nephrocalcinosis (HP:0000121); excluded: Nephrolithiasis (HP:0000787); excluded: Elevated circ